In [82]:
from copy import deepcopy
from glob import glob
from utils import read_yaml, read_json, write_yaml

foo = read_yaml("./train.yaml")

for capacity in [2, 4, 8, 16, 32, 64]:
    for pretrain_semantic in [True, False]:
        for seed in [0, 1, 2, 3, 4]:
            foo = deepcopy(foo)
            foo["pretrain_semantic"] = pretrain_semantic
            foo["capacity"] = {
                "episodic": capacity // 2,
                "semantic": capacity // 2,
                "short": 1,
            }
            foo["seed"] = seed
            write_yaml(foo, f"{capacity}_{pretrain_semantic}_{seed}.yaml")


In [1]:
from glob import glob
import os
import numpy as np
from utils import read_yaml, read_json

results_ = []
for path in glob("./lightning_logs/*/test_debug-*"):
    dir_name_old = "/".join(path.split("/")[:-1])
    hparams_path = os.path.join(dir_name_old, "hparams.yaml")
    hparams = read_yaml(hparams_path)
    des_size = hparams["des_size"]
    capacity = hparams["capacity"]
    capacity = capacity["episodic"] + capacity["semantic"]
    pretrain_semantic = hparams["pretrain_semantic"]
    gpus = hparams["gpus"]
    seed = hparams["seed"]

    test_mean = float(path.split("test_debug-mean=")[-1].split("-std")[0])
    results_.append((des_size, capacity, pretrain_semantic, seed, test_mean))

results = {}
for capacity in [2, 4, 8, 16, 32, 64]:
    for pretrain_semantic in [False, True]:
        results[f"capacity={capacity}_pretrain_semantic={pretrain_semantic}"] = round(np.mean(
            [
                foo[-1]
                for foo in results_
                if foo[1] == capacity and foo[2] == pretrain_semantic
            ]
        ), 3).item(), round(np.std(
            [
                foo[-1]
                for foo in results_
                if foo[1] == capacity and foo[2] == pretrain_semantic
            ]
        ).item(), 3)
results


{'capacity=2_pretrain_semantic=False': (-54.821, 6.35),
 'capacity=2_pretrain_semantic=True': (-62.357, 9.042),
 'capacity=4_pretrain_semantic=False': (-46.625, 4.526),
 'capacity=4_pretrain_semantic=True': (-46.245, 6.818),
 'capacity=8_pretrain_semantic=False': (-19.292, 10.413),
 'capacity=8_pretrain_semantic=True': (-21.65, 7.268),
 'capacity=16_pretrain_semantic=False': (24.252, 6.438),
 'capacity=16_pretrain_semantic=True': (14.194, 11.754),
 'capacity=32_pretrain_semantic=False': (51.839, 6.16),
 'capacity=32_pretrain_semantic=True': (70.289, 9.34),
 'capacity=64_pretrain_semantic=False': (77.515, 17.593),
 'capacity=64_pretrain_semantic=True': (78.726, 10.394)}

In [2]:
import logging

logger = logging.getLogger()
logger.disabled = True

from room_env.utils import print_handcrafted

print_handcrafted(
    env="RoomEnv-v2",
    des_size="l",
    seeds=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    question_prob=0.1,
    policies={
        "memory_management": "rl",
        "question_answer": "episodic_semantic",
        "encoding": "argmax",
    },
    capacities=[2, 4, 8, 16, 32, 64],
)

capacity=2, strategy=episodic,	pre_sem=False,	rewards_mean=-87, rewards_std=13
capacity=2, strategy=semantic,	pre_sem=False,	rewards_mean=-79, rewards_std=16
capacity=2, strategy=random,	pre_sem=False,	rewards_mean=-84, rewards_std=17
capacity=2, strategy=pre_sem,	pre_sem=True,	rewards_mean=-72, rewards_std=21

capacity=4, strategy=episodic,	pre_sem=False,	rewards_mean=-85, rewards_std=15
capacity=4, strategy=semantic,	pre_sem=False,	rewards_mean=-47, rewards_std=18
capacity=4, strategy=random,	pre_sem=False,	rewards_mean=-74, rewards_std=14
capacity=4, strategy=pre_sem,	pre_sem=True,	rewards_mean=-63, rewards_std=28

capacity=8, strategy=episodic,	pre_sem=False,	rewards_mean=-77, rewards_std=16
capacity=8, strategy=semantic,	pre_sem=False,	rewards_mean=-29, rewards_std=27
capacity=8, strategy=random,	pre_sem=False,	rewards_mean=-64, rewards_std=22
capacity=8, strategy=pre_sem,	pre_sem=True,	rewards_mean=-53, rewards_std=33

capacity=16, strategy=episodic,	pre_sem=False,	rewards_mean=-

In [ ]:
from glob import glob
import os
from utils import read_yaml

# des_size=l-capacity=2-pretrain=True-gpus=0-seed=0

for path in glob("./lightning_logs/version*/test_debug-*"):
    dir_name_old = "/".join(path.split("/")[:-1])
    hparams_path = os.path.join(dir_name_old, "hparams.yaml")
    hparams = read_yaml(hparams_path)
    des_size = hparams["des_size"]
    capacity = hparams["capacity"]
    capacity = capacity["episodic"] + capacity["semantic"]
    pretrain_semantic = hparams["pretrain_semantic"]
    gpus = hparams["gpus"]
    seed = hparams["seed"]
    dir_name_new = f"des_size={des_size}-capacity={capacity}-pretrain={pretrain_semantic}-gpus={gpus}-seed={seed}"
    print(dir_name_old, dir_name_new)
    os.rename(dir_name_old, dir_name_new)


In [ ]:
import gym
import room_env

capacity = 32
env = gym.make(
    "RoomEnv-v2",
    des_size="l",
    seed=0,
    policies={
        "memory_management": "rl",
        "question_answer": "episodic_semantic",
        "encoding": "argmax",
    },
    capacity={"episodic": capacity // 2, "semantic": capacity // 2},
    question_prob=0.1,
    observation_params="perfect",
    allow_random_human=True,
    allow_random_question=True,
    pretrain_semantic=False,
    check_resources=False,
)
state = env.reset()


# Plotting


## Average sum of rewards per episode, validation (5 random runs)


In [ ]:
from memory.plotting import plot_training_validation_results

plot_training_validation_results(
    data_dir="./data/",
    kind="val_total_reward_mean",
    capacity=32,
    save_dir="./figures/",
    ymin=64,
    ymax=128,
    ylog=False,
    xlabel="Step",
    figsize=(10, 10),
    legend_loc="lower right",
)

## Average training loss (5 random runs)


In [ ]:
plot_training_validation_results(
    data_dir="./data/",
    kind="train_loss",
    capacity=32,
    save_dir="./figures/",
    ymin=None,
    ymax=None,
    ylog=True,
    xlabel="Step",
    figsize=(10, 10),
    legend_loc="upper right",
)

## Average sum of rewards per episode, test (5 random runs), capacity=32


In [ ]:
from memory.plotting import plot_test_results

plot_test_results()

## Average sum of rewards per episode, test (5 random runs), capacity=32, all sizes


In [ ]:
from memory.plotting import plot_test_results_all_capacities
import matplotlib.pyplot as plt

plot_test_results_all_capacities()

## Load the q values


In [ ]:
from memory.utils import read_json

test_debug_scratch_path = "./models/des_size=l-capacity=32-pretrain=False-gpus=0-seed=1/test_debug-mean=115.0-std=3.225.json"

test_debug_scratch = read_json(test_debug_scratch_path)["0"]

test_debug_pretrained_path = "./models/des_size=l-capacity=32-pretrain=True-gpus=0-seed=2/test_debug-mean=117.5-std=2.54.json"
test_debug_pretrained = read_json(test_debug_pretrained_path)["0"]

for step in [86]:
    print(f"STEP: {step}")
    print()
    print(f"SCRATCH: {test_debug_scratch[step]}")
    print()
    print(f"PRETRAINED: {test_debug_pretrained[step]}")
    print()
    print(
        "----------------------------------------------------------------------------"
    )

## Load the trained models and visualize the t-SNE-reduced embeddings.


In [ ]:
from memory.plotting import UnderstandModel

um = UnderstandModel()

um.compute_reduction(
    tsne_params={
        "n_components": 2,
        "perplexity": 5,
        "init": "pca",
        "learning_rate": "auto",
    }
)

print(
    len(um.indices["humans"]),
    len(um.indices["objects"]),
    len(um.indices["object_locations"]),
)

for semantic in ["scratch", "pretrained"]:
    for interest in ["humans", "objects", "object_locations"]:
        um.plot_embeddings(semantic=semantic, interest=interest)